## xcube Generator Python Access Library

This notebook shows how to generate xcube cube using the xcube-gen web service [xcube-gen.brockmann-consult.de](https://xcube-gen.brockmann-consult.de).
Please be aware, this notebook will not run unless you have access to the xcube-gen service as well as a bucket on AWS.

In [1]:
from xcube_gen.jobapi import JobApi
api = JobApi()
api.whoami

'helge.dzierzon@brockmann-consult.de'

## Generate a config

In [3]:
cfg = {
  "input_configs": [
    {
      "store_id": "@sentinelhub",
      "data_id": "S2L2A",
      "open_params": {
        "tile_size": [
          1000,
          1000
        ]
      }
    }
  ],
  "cube_config": {
    "variable_names": [
      "B01",
      "B02"
    ],
    "bbox": [
      7,
      53,
      9,
      55
    ],
    "spatial_res": 0.001,
    "crs": "WGS84",
    "time_range": [
      "2000-06-20",
      "2000-06-22"
    ],
    "time_period": "1D"
  },
  "output_config": {
    "store_id": "s3",
    "store_params": {
      "bucket_name": "***",
      "aws_access_key_id": "***",
      "aws_secret_access_key": "***"
    }
  }
}

## Generating an xcube

In [4]:
api.create(cfg=cfg)

{'result': {'job_id': 'xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017',
  'status': {'active': None,
   'completion_time': None,
   'conditions': None,
   'failed': None,
   'start_time': None,
   'succeeded': None}},
 'status': 'ok'}

## Getting the Status of a Generation Job

In [5]:
api.status('xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017')

{'result': {'job_id': 'xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017',
  'output': ['Awaiting generator configuration JSON from TTY...',
   'Cube generator configuration loaded from TTY.',
   'Generating cube - ...',
   'Generating cube - 0.0%: Opening input(s) - ...',
   'Generating cube - 10.0%: Opening input(s) - 100.0%',
   'Generating cube - 10.0%: Opening input(s) - done.',
   'Generating cube - 10.0%: Resampling cube(s) - ...',
   'Generating cube - 15.0%: Resampling cube(s) - 50.0%',
   'Generating cube - 20.0%: Resampling cube(s) - 100.0%',
   'Generating cube - 20.0%: Resampling cube(s) - done.',
   'Generating cube - 20.0%: Writing output - ...',
   'Generating cube - 100.0%: Writing output - 100.0%',
   'Generating cube - 100.0%: Writing output - done.',
   'Generating cube - done.'],
  'status': {'active': 1,
   'completion_time': None,
   'conditions': None,
   'failed': None,
   'start_time': 'Thu, 02 Jul 2020 08:48:59 GMT',
   'succeeded': None}},
 'status': 'ok'}

## Listing my Jobs

In [6]:
api.list()

{'result': [{'job_id': 'xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017',
   'status': {'active': None,
    'completion_time': 'Thu, 02 Jul 2020 08:49:13 GMT',
    'failed': None,
    'start_time': 'Thu, 02 Jul 2020 08:48:59 GMT',
    'succeeded': 1}},
  {'job_id': 'xcube-gen-2f5793af-e27d-4768-a28e-6e731564eab7',
   'status': {'active': None,
    'completion_time': None,
    'failed': 2,
    'start_time': 'Thu, 02 Jul 2020 08:00:53 GMT',
    'succeeded': None}},
  {'job_id': 'xcube-gen-32bdad43-3f9a-4e4c-9264-3400e63bbe98',
   'status': {'active': None,
    'completion_time': None,
    'failed': 2,
    'start_time': 'Thu, 02 Jul 2020 07:58:53 GMT',
    'succeeded': None}},
  {'job_id': 'xcube-gen-5f5a9a4a-bc6f-4b8b-b7a4-0b15a8e9a6cd',
   'status': {'active': None,
    'completion_time': None,
    'failed': 2,
    'start_time': 'Thu, 02 Jul 2020 07:51:10 GMT',
    'succeeded': None}},
  {'job_id': 'xcube-gen-6cfdf2ec-b0b4-4108-962d-4cb3e43d9e35',
   'status': {'active': None,
    'compl

## Deleting a job

In [2]:
api.delete('xcube-gen-08bc03f4-c995-45d9-ae81-16e90fc03017')

{'result': 'Success', 'status': 'ok'}